In [5]:
import numpy as np
print("Creating environment")
#define constants
L = 4
N = 3

#read/write
#formDict = {}

#define macros (algebra.ngens())

#basic exterior algrebra functions
def expExt(form):
    return sum(form^i/factorial(i) for i in (0..N))
def pfaff (form):
    return ((form^(N)/factorial(N)).hodge_dual()).constant_coefficient()

#"2D pfaffian"
def pfaff2 (form):
    return ((form^(N*2)/factorial(N*2)).hodge_dual()).constant_coefficient()
def inverseExt (form):
    con = factorial(N-1)*pfaff(form)
    if(con == 0):
        return E.zero()
    else:
        return ((form^(N-1)).hodge_dual())/con

#compute inverse of form given pfaffian
def inverseExtPfaf (form, pfafform):
    if (pfafform == 0):
        return E.zero()
    else:
        return ((form^(N-1)).hodge_dual())/(factorial(N-1)*pfafform)

def hasInverse (form, invForm):
    #return (expExt(form)).hodge_dual() == pfaff(form)*expExt(invForm)
    return inverseExt(invForm) == form
def innerProd(form1, form2): #assuming orthonormal basis and non-negativity. This means that for decomposable standard forms, just take product of coefficients. For
    return vector(form1).dot_product(vector(form2))

#graph algorithms
def toGraphForm(form):
    G = Graph()
    G.weighted(True)
    L = list()
    for ter in form:
        G.add_vertex()
        L.append(set(ter[0]))
    for i in range(len(L)):
        for j in range(i):
            inter = L[i] & L[j]
            if(inter):
                G.add_edge(i,j,len(inter))
    #print(L)
    P = G.graphplot(edge_labels=True,)
    P.show()
    return G

#tbd
def graphToForm(Gp):
    incorpVertices = []
    for Vx in Gp.vertex_iterator():
        possib = E.gens()
        for Vy in incorpVertices:
            if (not Gp.has_edge(Vx,Vy)):
                for y in Gp.get_vertex(Vy):
                    possib.remove(y)
            #for Nx in Gp.neighbor_iterator(Vx):

    return True

#form and vector manipulations in coefficient-basis format
def formTerm(vector):
    return prod(E.gen(_) for _ in vector)

def toForm(lv):
    return sum(formTerm(_) for _ in lv)

#form and vector manipulations in coefficient format
def toVector(form):
    outy = copy(zeroVector)
    for ele in form:
        outy[combList.index(list(ele[0]))] = ele[1]
    return tuple(outy)

def vecToForm(vec):
    outy = E.zero()
    for i in range(len(vec)):
        if (vec[i] != 0):
            outy += vec[i]*formTerm(combList[i])
    return outy

#randomly assign signs to form in coefficient-basis format
def toRsignedForm(lv):
    return sum(choice([-1,1])*formTerm(_) for _ in lv)

#randomly assign signs and geometric value to form in coefficient-basis format
def toRsignedDGeoForm(lv):
    return sum(choice([-1,1])*np.random.geometric(p=geoP)*formTerm(_) for _ in lv)

#real distributions, need to change algebra before use
sigma = 1
T = RealDistribution('gaussian', sigma)
geoP = 0.4

def toRsignedGausForm(lv):
    return sum(choice([-1,1])*T.get_random_element()*formTerm(_) for _ in lv)


#define algebra
E.<e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11> = ExteriorAlgebra(QQ)
comb = Combinations(range(E.ngens()),L)
combList = comb.list()
zeroVector = [0] * len(combList)
comb2 = Combinations(range(E.ngens()),2)

baseVector = [[0,1,2,3],[4,5,6,7],[8,9,10,11]]
base2Vector = [[0,1], [2,3], [4,5], [6,7], [8,9], [10,11]]

combListBaseSub = copy(combList)
for _ in baseVector:
    combListBaseSub.remove(_)

baseForm = toForm(baseVector)
base2Form = toForm(base2Vector)
def RBaseForm():
    return toRsignedForm(baseVector)

#algorithms from wed 6/3

#produce a random signed form with rank
def randomForm(rank):
    outForm = baseForm
    for i in range(rank - 3):
        outForm += choice([-1,1])*prod(E.gen(j) for j in comb.random_element())
    return(outForm)

def random2Form(rank):
    return sum(choice([-1,1])*prod(E.gen(j) for j in comb2.random_element()) for i in range(rank))

def recurFindInversion(form, itera):
    iForm = inverseExt(form)
    boolInv = hasInverse(form, iForm)
    for i in range(itera):
        if (boolInv):
            return [form,i]
        else:
            form = inverseExt(iForm)
            iForm = inverseExt(form)
            boolInv = hasInverse(form, iForm)
    return [form, False]

def formDifference(form1, form2):
    vec1 = set()
    vec2 = set()
    for ele in form1:
        vec1.add(ele[0])
    for ele in form2:
        vec2.add(ele[0])
    out_ = E.zero()
    vec1 = vec1-vec2

    for ele in vec1:
        out_ += formTerm(ele)

    return out_

constantTries = 8

def getFormInfo(testE):
    vecE = toVector(testE)
    if not vecE in formDict:
        pfafform = pfaff(testE)
        invTestE = inverseExtPfaf(testE, pfafform)
        recurGenInform(testE, invTestE, pfafform, constantTries)
    return formDict[vecE]

#csv format: 0. vectorform 1. form hash 2. inverse hash 3. double inverse hash 4. rank  5. pfaffian 6. diff 7. turns until invertible (-1 if unstable, 0 if invertible) 8. tries
def recurGenInform(testE, invTestE, pfafform, tries):
    vecE = toVector(testE)
    if tries == 0:
        return -1
    elif vecE in formDict:
        return formDict[vecE][6]
    else:
        #find inverse
        #pfafform = pfaff(testE)
        #invTestE = inverseExtPfaf(testE, pfafform)
        vecInvE = toVector(invTestE )
        ranky = len(testE)
        tagInvE = 0
        tagDInvE = 0

        #check if inverse is in dict. if inverse is in dict, th
        if vecInvE in formDict:
            invValues = formDict[vecInvE]
            stability = invValues[6]+1
            if (stability == 0):
                stability = -1
            tagInvE = invValues[0]
            tagDInvE = invValues[1]
            vecDInvE = list(formDict)[tagDInvE]
            difff = invValues[3]-ranky

        else:
            #check if invertible. If invertible, create entry for inverse and halt. Else, recurgen
            #repetitive
            invPfafform = pfaff(invTestE)
            dtestE = inverseExtPfaf(invTestE, invPfafform)
            vecDInvE = toVector(dtestE)
            isInve = dtestE == testE
            rankInv = len(invTestE)
            if (isInve):
                stability = 0
                if (vecInvE != vecE):
                    tagDInvE = len(formDict)+1
                    tagInvE = len(formDict)
                     #might be simpler formula here
                    formDict[vecInvE] = [tagInvE, tagDInvE, tagInvE, rankInv, invPfafform, rankInv-ranky, 0, constantTries-tries]
            else: #recurgen inverses first, then copy
                stability = recurGenInform(invTestE, dtestE, invPfafform, tries-1)+1
                if (stability == 0):
                    stability = -1
                if vecInvE in formDict:
                    tagDInvE = formDict[vecInvE][1]
                    tagInvE = len(formDict)-1
                else:
                    tagDInvE = 0
                    tagInvE = 0

            difff = ranky - rankInv
        formHash = len(formDict)
        formDict[vecE] = [formHash, tagInvE, tagDInvE, ranky, pfafform, difff, stability, constantTries-tries]
        dictRanks.add(ranky)
        return stability

def regProd(form1, form2):
    return (form1.hodge_dual() * form2).hodge_dual()/N

def stringToVec(stry):
    outy = stry.strip(')(').split(',')
    outy = [QQ(i) for i in outy]
    return tuple(outy)

def randomBernVec(k):
    outvec = []
    for i in range(k):
        outvec += [choice([-1,0,1])]
    return outvec

def randomBernMatrix():
    outvec = []
    for i in range(N*L):
        outvec += [randomBernVec(N*L)]
    return Matrix(outvec)

print("Environment created")

Creating environment
Environment created


In [7]:
#read files. Run this regularly to save to CSV
#csv format: 0. vectorform 1. form hash 2. inverse hash 3. double inverse hash 4. rank  5. pfaffian 6. diff 7. turns until invertible (-1 if unstable, 0 if invertible) 8. tries

import csv
import sys
from collections import OrderedDict

csv.field_size_limit(sys.maxsize)

csvInfoName = "formsInfo.csv"
csvName0 = "forms0.csv"
csvName1 = "forms1.csv"

#determine which CSV is empty
activeCSV = csvName0
readCSV = csvName1
f = open(csvInfoName, 'rt')
try:
    reader = csv.reader(f)
    for row in reader:
        if row[0] == "backupCSV":
            if row[1] == csvName0:
                activeCSV = csvName1
                readCSV = csvName0
            else:
                activeCSV = csvName0
                readCSV = csvName1
            break
finally:
    f.close()
print(activeCSV)

#read in the read only csv
formDict = OrderedDict()
dictRanks = set()
f = open(readCSV, 'rt')
try:
    reader = csv.reader(f)
    for row in reader:

        formDict[stringToVec(row[0])] = [eval(row[1])]+[stringToVec(row[2])]+[stringToVec(row[3])]+[eval(row[4])]+[eval(row[5])]+[eval(row[6])]+[eval(row[7])]
finally:
    f.close()
len(formDict)

forms0.csv


0

1
2
2
2
2
2
2
2
3


In [10]:
#double-bernouli form info generation by rank. Generating combT also takes a while, so recommend large itera
itera = 100
startRange = 2
endRange = 30
incrmnt = 2

for j in range(startRange, endRange, incrmnt):
    print(j)
    combT = Combinations(combListBaseSub,j)

    for i in range(itera):
        testComb = combT.random_element()
        #print(testComb)
        testE = toRsignedForm(testComb)+RBaseForm()
        tempp = getFormInfo(testE)

#write to active CSV
writerDict = {}
# f = open(activeCSV, 'wt')
# try:
#     writery = csv.writer(f)
#     writery.writerow(["Hello World"])
# finally:
#     f.close()

print(1)
for i in dictRanks:
    f = open(str(i)+activeCSV, 'wt')
    writerDict[i] = [f, csv.writer(f)]

print(2)
try:
    for ele in formDict:
        ranky = formDict[ele][3]
        #print(ranky)
        writerDict[ranky][1].writerow([ele]+formDict[ele])
finally:
    print(2)
    for ele in writerDict:
        writerDict[ele][0].close()
        print(2)
print(3)
#now clear the read CSV
for i in dictRanks:
    open(str(i)+readCSV, 'w').close()

#print the new dictionary info
f = open(csvInfoName, 'wt')
try:
    writer = csv.writer(f)
    writer.writerow(["Read me on the forms database"])
    writer.writerow(["The labels for the columns are as follows: 0. vectorform 1. form hash 2. inverse hash 3. double inverse hash 4. rank  5. pfaffian 6. difference in rank between inverse and form 7. pseudo inverses taken until stable (-1 if unstable) 8. tries at stabilizing so far"])
    writer.writerow(["backupCSV", activeCSV])
    writer.writerow(["The inverse hash is 0 if the inverse is not generated"])
finally:
    f.close()
# open(readCSV, 'w').close()

2
4
6
8
10
12
14
16
18
20
22
24
26
28
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
